In [28]:
import numpy as np
from numpy import random
import pdb
import copy
from time import sleep
import pandas as pd



Game should start with 9 tiles

these 9 cards always contain at least one 3 and at most 4 of either color. (4 ones,4, twos, 1 three)

The highest number you can draw is 3 ranks below your high card. If your high card is 24 or less, you only draw 3s. With a high card of 48, 6s start to show up. At 96, 12s come in and at 192, 24s appear.


The probability for each possible bonus card value seems to be equal. This means that as your high card increases it is more and more unlikely that you will see the highest possible bonus card. On the flip side, the value of bonus cards (even if they are announced in the coming update) becomes less and less predictable as the high card increases.


• The game uses an undrawn stack of 12 cards (four 1s, four 2s and four 3s). A stack is created at the beginning of a game and cards are drawn individually both to fill the board (see above) and to be added with every move. When a stack is depleted, a new one is created. I know I posited this theory before and then retracted it but it just fits to many beginnings of sequences (where I suspect the error-rate is lower) and I still believe in it. This has some consequences:


• I am not sure when a new high card takes effect. It either does so immediately (during a running stack) or at the start of the next stack (see below on the topic of stacks). Figuring this out is particularly nasty due to the problems listed in section 1 and the very low probability that you see an appropriate bonus card after the high card increases but before a new stack begins.


It could just as well be possible that bonus cards are completely unrelated to stacks. Instead, there is just a constant 5 % chance on every move that your next card is a bonus card

In [29]:
class Tiles:
    def __init__(self):
        
        self.possible_icons  = np.array([1,2,3,6,12,24,48,96,192,384,768,1536,3072,6144, 12288])
        self.tile_values  = np.array([0, 0, 3, 9, 27, 81, 243, 729, 2187, 6561, 19683, 59049, 177147, 531441, 1500000])
        self.score = 0;
#         self.icons = dict()
#         self.values = dict()
#         pos = 0
#         for index in self.possible_icons:
            
#             self.icons[index] =pg.image.load('C:/Henry/PythonProjects/Pygame/Threes/images/_' + str(index) + '.png')
#             self.values[index] =self.tile_values[pos]
#             pos +=1
        self.stack = np.array([1,1,1,1,2,2,2,2,3])
#         self.stack = self.stack[random.permutation(9)]    
            
        self.grid = np.zeros((16,),dtype = int)
        self.grid[0:9] = self.stack
        self.grid = self.grid[random.permutation(16)]
        self.grid =  np.reshape(self.grid,(4,4))
        self.move = np.zeros((4,4),dtype = bool)
        self.game_over = False
        self.bonus_tiles = []
        
    def __copy__(self):
        return Tiles()
    
    def determineBonusTiles(self):
        current_max = np.max(self.grid)
        
        max_position  = np.argwhere(self.possible_icons == current_max)
        self.bonus_tiles = self.possible_icons[4:max_position[0][0]]
        
        return self
        
    def createStack(self):
        
        self = self.determineBonusTile()
        
        self.stack = np.array([1,1,1,1,2,2,2,2,3])
        
        if self.bonus_tiles.shape[0]>0:
            if random.rand(1)>.5:
                r =random.randint(0,len(bonus_cards))
                self.stack = np.append(self.stack,self.bonus_tiles[r])
        rand_perm =  random.permutation(self.stack.shape[0])
        self.stack = self.stack[rand_perm]
        
        return self
        
    def drawStack(self):
        pos =0;
        for tile in self.stack:
            current_x = 400+ self.icons[tile].get_height()*np.floor(pos/4)
            current_y = self.icons[tile].get_height()*np.mod(pos,4)
            screen.blit(self.icons[tile],(current_x,current_y))
            
            pos +=1
                
            


    
    def determineMove(self,keys):
        current_move = [0, 0]
   
        if keys[pg.K_LEFT]:
            current_move = [0, -1]
  
        elif keys[pg.K_UP]:
            current_move = [-1, 0]
          
        elif keys[pg.K_RIGHT]:
            current_move = [0, 1]
            
        elif keys[pg.K_DOWN]:
            current_move = [1,0]
            
        return current_move
    def updatePosition(self,current_move):
        self.move = np.zeros((4,4),dtype = bool)
        x_range = np.array(range(4),dtype = int)
        y_range = np.array(range(4),dtype = int)
        if current_move[0] == 1:
            x_range = np.flip(x_range)
        if current_move[1] == 1:
            y_range = np.flip(y_range)
        
  #there only need to be 1 True per row/col in self.move. If the player moves right, then the first tile to move (col 3 to 0)
#should be True, the rest slide over, but this is redundant
        for x in x_range:
            if x+current_move[0]>-1 and x+current_move[0]<4: #if the move does not take you off the grid
                for y in y_range:
                   
                    if y+current_move[1]>-1 and y+current_move[1]<4: #if the move does not take you off the grid
                        if self.grid[x,y] !=0: #if the current tile is not empty
                            new_pos = np.array([x,y]) + current_move #calculate where the current move will take this tile
                            if self.grid[new_pos[0],new_pos[1]] == 0: #if the new location is empty
                                self.grid[new_pos[0],new_pos[1]]  = self.grid[x,y] #fill new location with tile
                                self.grid[x,y] = 0 #empty previous location
                                if current_move[0] !=0:
                                    self.move[x_range[-1],y] = True #record move
                                elif current_move[1] !=0:
                                    self.move[x,y_range[-1]] = True #record move
                                
                            elif self.grid[new_pos[0],new_pos[1]] + self.grid[x,y] == 3: #if move combines  1 and 2
                                self.grid[x,y] =0 #empty old position
                                self.grid[new_pos[0],new_pos[1]]  = 3
                                if current_move[0] !=0:
                                    self.move[x_range[-1],y] = True #record move
                                elif current_move[1] !=0:
                                    self.move[x,y_range[-1]] = True #record move
                               
                            elif self.grid[new_pos[0],new_pos[1]] == self.grid[x,y]:
                                if self.grid[new_pos[0],new_pos[1]] != 2:
                                    if self.grid[new_pos[0],new_pos[1]] + self.grid[x,y]>3:
                                    
                                        self.grid[new_pos[0],new_pos[1]]  = self.grid[x,y]*2
                                        self.grid[x,y]=0
                                        if current_move[0] !=0:
                                            self.move[x_range[-1],y] = True #record move
                                        elif current_move[1] !=0:
                                            self.move[x,y_range[-1]] = True #record move
        
                                    
                            

        return self
    
    def addTile(self,current_move):
        if np.sum(self.move,axis = None)>0:
      
            if current_move[0] ==-1:
                empty_spaces =  self.grid[3,:]==0
                moved_spaces =  np.sum(self.move,axis = 0)

            elif current_move[0] ==1:
                empty_spaces =  self.grid[0,:]==0
                moved_spaces =  np.sum(self.move,axis = 0)

            elif current_move[1] ==-1:
                empty_spaces =  self.grid[:,3]==0
                moved_spaces =  np.sum(self.move,axis = 1)

            elif current_move[1] ==1:
                empty_spaces =  self.grid[:,0]==0
                moved_spaces =  np.sum(self.move,axis = 1)

            new_spaces = empty_spaces * moved_spaces
            new_spaces = np.nonzero(new_spaces)
            new_spaces=new_spaces[0]

            if len(new_spaces)>0:
                r = np.random.permutation(len(new_spaces))
                new_spaces = new_spaces[r[0]]
                if current_move[0] == -1:
                    self.grid[3,new_spaces] = self.stack[0]
                elif current_move[0] == 1:
                    self.grid[0,new_spaces] = self.stack[0]
                elif current_move[1] == -1:
                    self.grid[new_spaces,3] = self.stack[0]
                elif current_move[1] == 1:
                    self.grid[new_spaces,0] = self.stack[0]
                  
                else:
                    print('No tile added!!!!!!!')
           
            else:
                pdb.set_trace()
                
            if self.stack.shape[0]>1:
                self.stack = np.delete(self.stack,0)
            else:
                print('created new stack')
                self =  self.createStack()
        else:
            self.game_over = True
            print('no move possible... end of game!')
            pdb.set_trace()
        return self
        
    def drawTile(self):
        for x in range(4):
            for y in range(4):
                if self.grid[x,y] !=0:
                    current_number = self.grid[x,y]
                    if current_number in self.icons:
                        screen.blit(self.icons[current_number],(y*self.icons[current_number].get_width(),
                                      x*self.icons[current_number].get_height()))
                    else:
                        print('Invalid current_number = ' + str(current_number))
                        running = False
                    
                    
    def calculateScore(self):
        self.score =0;
        for x in range(4):
            for y in range(4):
                current_number = self.grid[x,y]
                if current_number>0:
                    self.score = self.score + self.tile_values[current_number]
        return self

        
        
   
    

In [30]:
class HiveMind(Tiles):
    def __init__(self):
        Tiles.__init__(self)
        self.current_move = np.array([0,0],dtype = int)
        self.possible_grids = [];
    
    def possibleNewGrids(self):
        
#       we will use the moves grid to set the unknown positions to a special character to denote that 
#       the new value at this location is unknown
        new_grid = np.where(self.move,-1,self.grid)
        
#       possible_moves = np.sum(self.move, axis = None)
        
        #caclulate all the possible new grids there are. This
#       depends on (1) how many tiles emptied (Et) and (2) what new tiles are possible (Tp). HM
#       will know the following distribution of new cards: 1 = 0.4225, 2 = .4225,  3 = 0.105, bonus (Bt) = 0.05 
#       
#       for each Et, a new grid containing all possible Tp should be calculated.
#       Since only one new card is added, the possible new grids is length(Et)*length(Tp)

        self = self.determineBonusTiles()
        Et_index = np.where(new_grid==-1)
        
        
        Tp = [1,2,3]
        if len(Et_index)>0:
            Tp = np.append(Tp,self.bonus_tiles)


        
        x_list = Et_index[0]
        y_list = Et_index[1]
        self.possible_grids = []
        w = np.where(self.grid==0,True,False)
        for index in range(len(x_list)):
            x = x_list[index]
            y = y_list[index]
            for tile_index in range(len(Tp)):
                #loop through to create all the possible new grids,
#                 placing all the Tp in each Et
                current_grid = copy.copy(self.grid)
                current_grid[x,y] =Tp[tile_index]
                self.possible_grids.append(current_grid)
        return self

In [44]:
game_tiles = Tiles()
hm = HiveMind()
hm.grid = copy.copy(game_tiles.grid)



#determine next move
print(game_tiles.grid)

move_list = [[-1,0],[0,1],[1,0],[0,-1]]
original_grid = copy.copy(hm.grid)
possible_grids=[]
empty_tiles=[]
move_result=[]
for move in move_list:
    hm.grid = copy.copy(original_grid)
    hm.updatePosition(move)
    empty_tiles.append(np.sum(np.where(hm.grid==0,1,0)))
    move_result.append(hm.grid)
    hm.possibleNewGrids()    
    for grid in hm.possible_grids:
        empty_tiles2=[]
        for move2 in move_list:
            hm.grid = copy.copy(grid)
            hm.updatePosition(move)
            empty_tiles2.append(np.sum(np.where(hm.grid==0,1,0)))
        pdb.set_trace()
            
        



print(empty_tiles)




    
    
    
    




     

    
      
            

[[1 1 0 2]
 [0 2 0 2]
 [0 1 0 3]
 [1 0 2 0]]
> <ipython-input-44-9f27b9c99fb2>(21)<module>()
-> for grid in hm.possible_grids:
(Pdb) empty_tiles2
[7, 7, 7, 7]
(Pdb) empty_tiles[0]
8
(Pdb) c
> <ipython-input-44-9f27b9c99fb2>(21)<module>()
-> for grid in hm.possible_grids:
(Pdb) empty_tiles2
[7, 7, 7, 7]
(Pdb) c
> <ipython-input-44-9f27b9c99fb2>(21)<module>()
-> for grid in hm.possible_grids:
(Pdb) empty_tiles2
[7, 7, 7, 7]
(Pdb) q


BdbQuit: 

In [41]:
move_result[0]

array([[0, 0, 0, 0],
       [0, 1, 2, 0],
       [1, 1, 3, 0],
       [2, 3, 2, 0]])

In [53]:
4*(16**2)

1024

In [56]:
2*3/4

1.5